In [1]:
#! pip install geemap
#! pip install epiweeks

In [2]:
import ee
import geemap

In [3]:
# If you are working with a shape file stored in google drive remove the quotes (""" """)

"""
#connect with drive
from google.colab import drive
drive.mount('/content/drive')
"""

"\n#connect with drive\nfrom google.colab import drive\ndrive.mount('/content/drive')\n"

In [4]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

Enter verification code: 4/1AX4XfWiLzxjza4fmcgdNAU5zRgHvdvdkdlX04PVF9XQP-sLuvDEOprg5BRQ

Successfully saved authorization token.


In [34]:
Map = geemap.Map()
#Map

In [62]:
# https://sites.google.com/site/seriescol/shapes
shape = ee.FeatureCollection('users/dsrestrepo/mpio')

# Cauca = shape.filter(ee.Filter.eq('DPTO_CCDGO', 19))
# region = shape.filter(ee.Filter.eq('MPIO_CCNCT	', 19001))
# Medellin = ee.FeatureCollection('users/dsrestrepo/Medellin')

region = shape.filter(ee.Filter.eq('MPIOS', '19001'))
region_original = region
Map.addLayer(region, {}, "Region")

In [63]:
import numpy as np
def get_rectangular_region(region):
    coordenates = np.array(region.geometry().getInfo()['coordinates'][0])
    latitude = []
    longitude = []
    
    latitude = [coordenates[:,0].min(), coordenates[:,0].max()]
    longitude = [coordenates[:,1].max(), coordenates[:,1].min()]
    
    
    # Build square:
    coordinates_square = []
    '''
    for i in range(0,2):
        for j in range(0,2):
            coordinates_square.append([latitude[i],longitude[j]])
    '''
    
    coordinates_square.append([latitude[0],longitude[0]])
    coordinates_square.append([latitude[0],longitude[1]])
    coordinates_square.append([latitude[1],longitude[1]])
    coordinates_square.append([latitude[1],longitude[0]])
    coordinates_square.append([latitude[0],longitude[0]])
    
    return coordinates_square

coordinates_square = get_rectangular_region(region)
print(coordinates_square)

[[-76.82029652569813, 2.5805247323487936], [-76.82029652569813, 2.35643096165237], [-76.44899703783351, 2.35643096165237], [-76.44899703783351, 2.5805247323487936], [-76.82029652569813, 2.5805247323487936]]


In [64]:
region = ee.Geometry.Polygon(
  [
    coordinates_square
  ]
)

Map.addLayer(region, {}, "Region_square")

In [65]:
Map

Map(bottom=517286.0, center=[2.506143229744594, -76.65401458740236], controls=(WidgetControl(options=['positio…

In [66]:
# Define the range of years
initial_year = 2015
end_year = 2020
years = ee.List.sequence(initial_year, end_year)
# Define the range of weeks:

first_2015_week = 44
weeks_2015 = list(range(first_2015_week,53)) # We don't have data till Half October, so we start in November 1st (epiweek 44)
weeks = list(range(1,53)) # 52 epiweeks each year

years.getInfo()
#months.getInfo()

[2015, 2016, 2017, 2018, 2019, 2020]

# Get number of images each month

In [67]:
from epiweeks import Week, Year

def images_per_week(year, week):
    
    # Get date from epiweek
    date = Week(year, week).startdate()
    
    # From date take year, month and day:
    year = date.year
    month = date.month
    day = date.day
    
    # range of dates to take sentinel data
    start_date = ee.Date.fromYMD(year, month, day)
    end_date = start_date.advance(7, "days")
    
    #https://developers.google.com/earth-engine/datasets/catalog/sentinel-2
    collection = ee.ImageCollection('COPERNICUS/S2') \
        .filterDate(start_date, end_date) \
        .filterBounds(region) \
        #.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 80))
    
    print(f'the images in epiweek {week} of year {year} are {collection.size().getInfo()}')
    # Take month and year of month with 0 images
    if collection.size().getInfo() == 0:
        print('is zero')
        return year, week

In [ ]:
dates_0_images = []
# See images by month
for year in years.getInfo():
    if year == 2015:
        for week in weeks_2015:
            dates_0_images.append(images_per_week(year, week))
    else:
        for week in weeks:
            dates_0_images.append(images_per_week(year, week))
dates_0_images = list(filter(None, dates_0_images))

the images in epiweek 44 of year 2015 are 0
is zero
the images in epiweek 45 of year 2015 are 0
is zero
the images in epiweek 46 of year 2015 are 0
is zero
the images in epiweek 47 of year 2015 are 4
the images in epiweek 48 of year 2015 are 0
is zero
the images in epiweek 49 of year 2015 are 3
the images in epiweek 50 of year 2015 are 0
is zero
the images in epiweek 51 of year 2015 are 6
the images in epiweek 52 of year 2015 are 2
the images in epiweek 1 of year 2016 are 0
is zero
the images in epiweek 2 of year 2016 are 2
the images in epiweek 3 of year 2016 are 6
the images in epiweek 4 of year 2016 are 2
the images in epiweek 5 of year 2016 are 0
is zero
the images in epiweek 6 of year 2016 are 2
the images in epiweek 7 of year 2016 are 2
the images in epiweek 8 of year 2016 are 0
is zero
the images in epiweek 9 of year 2016 are 0
is zero
the images in epiweek 10 of year 2016 are 2
the images in epiweek 11 of year 2016 are 0
is zero
the images in epiweek 12 of year 2016 are 4


In [ ]:
# List of weeks with 0 images
dates_0_images

# Cloud Mask

In [ ]:
# Function to mask clouds using the Sentinel-2 QA band
# param {ee.Image} image Sentinel-2 image
# return {ee.Image} cloud masked Sentinel-2 image

def maskS2clouds(image):
    qa = image.select('QA60')

    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11

    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cirrusBitMask).eq(0) \
      #.And(qa.bitwiseAnd(cloudBitMask).eq(0))

    return image.updateMask(mask).divide(10000)


# Create one image per month with as few clouds as possible

In [ ]:
from epiweeks import Week, Year

def weekly_image(year, week):
    
    # Get date from epiweek
    date = Week(year, week).startdate()
    
    # From date take year, month and day:
    year = date.year
    month = date.month
    day = date.day
    
    # Define Start Date
    start_date = ee.Date.fromYMD(year, month, day)

    # If no images in that week
    if ((year, week) in dates_0_images):
        print('in')
        # Take more days after week ends (Two weeks more after)
        #end_date = start_date.advance(15, "days")
        end_date = start_date.advance(21, "days")

        # Take more days before week starts (Two weeks before)
        start_date = start_date.advance(-7, "days")
    else:
        # ends two weeks after
        # end_date = start_date.advance(7, "days")
        end_date = start_date.advance(15, "days")
    
    # Take data and filter by dates and region
    #https://developers.google.com/earth-engine/datasets/catalog/sentinel-2
    collection = ee.ImageCollection('COPERNICUS/S2') \
        .filterDate(start_date, end_date) \
        .filterBounds(region)
    
    
    
    # Sentinel Images Composite
    #collection = collection.map(maskS2clouds)
    
    # Get a least cloudy image
    #image = ee.Image(collection.sort('CLOUDY_PIXEL_PERCENTAGE').first())
    
    # Composite 
    image = collection.median()
    
    return image
    
    

# Get image dimensions

In [ ]:
# Take a random image of dataset
test_year = 2017
test_week = 35
image = ee.Image(weekly_image(test_year, test_week))


In [ ]:
import numpy as np
# Mask image
image = image.clip(region).unmask()

#https://gis.stackexchange.com/questions/350771/earth-engine-simplest-way-to-move-from-ee-image-to-array-for-use-in-sklearn/351177#351177
# Reproject image from 15m/px -> 120m/px due to the maximum number of bits allowed in the array.
# We select just one band a reference (red)
image = image.reproject(crs = image.select('B4').projection(), scale= 120)
# Sample image as rectangle

#band_arrs = image.sampleRectangle(region=region.geometry(), defaultValue= 0.0)
band_arrs = image.sampleRectangle(region=region_original.geometry(), defaultValue= 0.0)


# take red band from array
band_arr_red = band_arrs.get('B4')
# Array to numpy
np_arr_red = np.array(band_arr_red.getInfo())
# Shape of image will be aprox:
height = (10 * np_arr_red.shape[0]) - 10
width = (10 * np_arr_red.shape[1]) - 10
print(f'the dimensions of the image will be height: {height} and width: {width}')

# Download RGB Image

In [ ]:
# https://www.usgs.gov/faqs/what-are-band-designations-landsat-satellites?qt-news_science_products=0#qt-news_science_products
# B4: Red, B3: Green, B2: Blue

vis_params = {'bands': ['B4',  'B3',  'B2'],
              'min': 0,   
              'max':255,}          
#              'max': 0.3,}
#              'max': 3500,}

In [ ]:
from functools import partial

# For the example just use till 2020
initial_year = 2016
end_year = 2016
years = ee.List.sequence(initial_year, end_year)

# For each year
for year in years.getInfo():
    
    # Create a new empty list of images
    images = ee.List([])
    
    if (year < 2015 or year > 2020):
        print('No images for that date')
        break
    # 2015 Starts after week 44
    if year == 2015:
        # Bound weekly_image function with the year
        bound_weekly_image = partial(weekly_image, year)
        # Apply function to each week
        for week in weeks_2015:
            # Get a single image each week
            images = images.add(bound_weekly_image(week))     
    else:
        # Bound weekly_image function with the year
        bound_weekly_image = partial(weekly_image, year)
        # Apply function to each week
        for week in weeks:
            images = images.add(bound_weekly_image(week))   
            

    for index in range(0, 52):
        # Ignore the last 43 index from 2015 because data starts in week 44
        if (year == 2015 and index < (first_2015_week - 1)):
            continue
        
        if (year == 2015):
          # Take form ee list the image
          image = ee.Image(images.get(index - (first_2015_week-1)))
        else:
          # Take form ee list the image
          image = ee.Image(images.get(index))
        
        # Name
        layer_name = "Image " + str(year) + "_" + str(index + 1)
        # 2015 starts in week 44
        if (year == 2015):
            layer_name = "Image " + str(year) + "_" + str(index + 1)
        
        # Download:
        ## To PC

        '''
        .map(function(image) {
         return image.multiply(512).uint8();
        }
        '''
        
        image_to_png = image.select(['B4', 'B3', 'B2']).divide(3500).multiply(255).toUint8()
        image_unmask = image_to_png.clip(region).unmask()

        geemap.get_image_thumbnail(image_unmask, 'ImagesSentinel/'+ layer_name + '.png', vis_params, dimensions=(width ,height), format='png', region=region) #region=region.geometry())

        ## To Drive
        # Export the image, specifying scale and region.
        # https://developers.google.com/earth-engine/guides/reducers_reduce_region
        #geemap.ee_export_image_to_drive(sharpened, description= layer_name + '_sharpend', 
        #                                folder='GEE/Medellin_Sentinel', 
        #                                region=region.geometry(), 
        #                                scale=10)

        '''
        task = ee.batch.Export.image.toDrive(**{
            'image': image.select(['B4', 'B3', 'B2', 'B5', 'B6', 'B7', 'B8']).divide(3500).multiply(255).toUint8(),
            'description': layer_name,
            'folder':'GEE/Medellin_Sentinel',
            'scale': 10,
            'region': region,

            'formatOptions': {'cloudOptimized': True},
            #'region': region.geometry()
        })

        task.start()
        '''

In [ ]:
#task.status()

# See an example of the images

In [60]:
from functools import partial

# For the example just use 2016
initial_year = 2016
end_year = 2016
years = ee.List.sequence(initial_year, end_year)

# For each year
for year in years.getInfo():
    
    # Create a new empty list of images
    images = ee.List([])
    
    if (year < 2015 or year > 2020):
        print('No images for that date')
        break
    # 2015 Starts after week 44
    if year == 2015:
        # Bound weekly_image function with the year
        bound_weekly_image = partial(weekly_image, year)
        # Apply function to each month
        for week in weeks_2015:
            # Get a single image each month
            images = images.add(bound_weekly_image(week))     
    else:
        # Bound weekly_image function with the year
        bound_weekly_image = partial(weekly_image, year)
        # Apply function to each month
        for week in weeks:
            images = images.add(bound_weekly_image(week))   
            
    #for index in range(0, 52):
    for index in range(0, 12):
        # Ignore the last 43 index from 2015 because data starts in week 44
        if (year == 2015 and index > first_2015_week):
            continue
        
        # Take form ee list the image
        image = ee.Image(images.get(index))
        
        # Name
        layer_name = "Image " + str(year) + "_" + str(index + 1)
        # 2015 starts in week 44
        if (year == 2015):
            layer_name = "Image " + str(year) + "_" + str(index + 44)

        image = image.clip(region).divide(3500).multiply(255).toUint8()
        
        Map.addLayer(image, vis_params, layer_name)
        

Map.center_object(region, 13)
Map

in
in
in
in
in
in
in
in
in
in
in
in


Map(bottom=1131.0, center=[2.4684876403634086, -76.63464678176533], controls=(WidgetControl(options=['position…